In [1]:
import sys, os, git, time

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from InitializeSimulation import InitializeSimulation
from Utilities import Utilities
from Outputs import Outputs
from MolecularDynamics import MolecularDynamics
from MonteCarlo import MonteCarlo

import numpy as np

In [8]:
self = MonteCarlo(number_atoms=[100, 400],
    epsilon=[0.1, 1.0], # kcal/mol
    sigma=[3, 6], # A
    atom_mass=[1, 1], # g/mol
    # atom_charge=[0], # in elementary charge units
    Lx=500, # A
    Ly=500, # A
    Lz=500, # A
    minimization_steps=0,
    maximum_steps=0,
    desired_temperature=100,
    desired_pressure=1,
    thermo = 250,
    dump = 250,  
    #tau_temp = 100, # fs
    #tau_press= 1000, # fs
    time_step=1, # fs
    seed=219817,
    data_folder = "mccode-output/")
self.run()
# run lammps for comparison
#os.system("/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial -in input.lammps > /dev/null")

TypeError: object.__init__() takes exactly one argument (the instance to initialize)

### Case 1

In [ ]:
ti = time.time()

def calculate_r(self, position_i, positions_j):
    """Calculate the shortest distance between position_i and positions_j.
    # to fix : use the MDAnalysis option
    """
    rij = (np.remainder(position_i - positions_j
                        + self.box_size/2., self.box_size) - self.box_size/2.)
    return np.linalg.norm(rij, axis=1)

energy_potential = 0
for position_i, sigma_i, epsilon_i in zip(self.atoms_positions,
                                            self.atoms_sigma,
                                            self.atoms_epsilon):
    r = self.calculate_r(position_i, self.atoms_positions)
    sigma_j = self.atoms_sigma
    epsilon_j = self.atoms_epsilon
    sigma_ij = np.array((sigma_i+sigma_j)/2)
    epsilon_ij = np.array((epsilon_i+epsilon_j)/2)
    energy_potential_i = np.sum(4*epsilon_ij[r>0]*(np.power(sigma_ij[r>0]/r[r>0], 12)-np.power(sigma_ij[r>0]/r[r>0], 6)))
    energy_potential += energy_potential_i

tf = time.time()
print(energy_potential/2, np.round(tf-ti,3))

-6.30730313864524e-22 0.114


### Case 2

In [ ]:
import matplotlib.pyplot as plt
import MDAnalysis as mda
from MDAnalysis import analysis
from MDAnalysis.analysis import distances

In [ ]:
epsilon_ij = []
for i in range(self.total_number_atoms):
    for j in range(i + 1, self.total_number_atoms):
        epsilon_i = self.atoms_epsilon[i]
        epsilon_j = self.atoms_epsilon[j]
        epsilon_ij.append((epsilon_i+epsilon_j)/2)
epsilon_ij = np.array(epsilon_ij)
sigma_ij = []
for i in range(self.total_number_atoms):
    for j in range(i + 1, self.total_number_atoms):
        sigma_i = self.atoms_sigma[i]
        sigma_j = self.atoms_sigma[j]
        sigma_ij.append((sigma_i+sigma_j)/2)
sigma_ij = np.array(sigma_ij)

box = np.array([self.box_size[0], self.box_size[1], self.box_size[2], 90, 90, 90])

ti = time.time()

r_ij = mda.analysis.distances.self_distance_array(self.atoms_positions, box)

energy_potential = np.sum(4*epsilon_ij*(np.power(sigma_ij/r_ij, 12)-np.power(sigma_ij/r_ij, 6)))

tf = time.time()
print(energy_potential, np.round(tf-ti,3))

-6.307298880441376e-22 0.018


[166.66666666666666, 166.66666666666666, 166.66666666666666, 90, 90, 90]

In [ ]:
np.vstack([box_size, box_geometry])

array([[166.66666667, 166.66666667, 166.66666667],
       [ 90.        ,  90.        ,  90.        ]])